# Miscellaneous Subset Selection

In this chapter, a few more methods for subset selection are described. The methods used in this chapter do not add any additional functionality to pandas, but are covered for completeness.

Believe it or not, there are still a few more ways to select subsets of data. I personally do not use the methods described in this chapter as each one of them provides no more functionality over the previously covered methods. These methods are presented for completeness. They are all valid syntax and many pandas users do actually use them so you may find them valuable.

In [ ]:
import pandas as pd
bikes = pd.read_csv('../data/bikes.csv')
bikes.head(3)

## The `query` method

The `query` method allows you to make boolean selections by writing the filter as a string. For instance, you would pass the string `'tripduration > 1000'` to select all rows of the `bikes` dataset that have a `tripduration` less than 1000. Let's see this command now.

In [ ]:
bikes.query('tripduration > 1000').head(3)

### Less syntax and more readable

The `query` method generally uses less syntax than boolean selection and is usually more readable. For instance, to reproduce the above with boolean selection in a single line would look like the following:

```
bikes[bikes['tripduration'] > 1000]
```

This looks a bit clumsy with the name `bikes` written twice right next to one another.

### Use strings `and`, `or`, `not`

Unlike boolean selection, you can use the strings `and`, `or`, and `not` instead of the operators which further aides readability with `query`. Let's select `tripduration` greater than 1000 and `temperature` greater than 85.

In [ ]:
bikes.query('tripduration > 1000 and temperature > 85').head(3)

### Use the `@` symbol to reference a variable name

By default, all words within the query string will attempt to reference the column name. You can, however, reference a variable name by preceding it with the `@` symbol. Let's assign the variable `x` to 5000 and reference it in a query.

In [ ]:
x = 5000
bikes.query('tripduration > @x').head(3)

### Reference strings with quotation marks

If you would like to reference a literal string within a query, you need to wrap it in quotes, or else pandas will attempt to use it as a column name. Let's select all 'Female' riders.

In [ ]:
bikes.query('gender == "Female"').head(3)

### Use 'in' for multiple equalities

You can query for multiple equalities with the word 'in' within your query like this:

In [ ]:
bikes.query('events in ["snow", "rain"]').head(3)

There are multiple syntaxes for the above that all work the same.

* `bikes.query('["snow", "rain"] in events')`
* `bikes.query('["snow", "rain"] == events')`
* `bikes.query('events == ["snow", "rain"]')`

### Use 'not in' to invert the condition

You can invert the result of an 'in' clause by placing the word 'not' before it.

In [ ]:
bikes.query('events not in ["cloudy", "partlycloudy", "mostlycloudy"]').head(3)

### Using the index with `query`

You can even use the word `index` to make comparisons against the index as if it were a normal column. Here, we select only the `events` that were 'cloudy' for an index value greater than 4000.

In [ ]:
bikes.query('index > 4000 and events =="cloudy" ').head(3)

### Use multiple comparison operators in a row
You can test that a column is contained between two values without using 'and'. Place the column name between the two less than (or greater than) signs as is done below.

In [ ]:
bikes.query('5000 < tripduration < 6000').head(3)

### Why I avoid query

The `query` method appears to provide a more readable approach to filtering our data based on the values, but it currently lacks the ability to reference column names with spaces. For instance, if we had a column name of 'trip duration', then we would have no way to reference it with `query`.

Using boolean selection as shown in previous chapters works for every situation, so I only use it. There has been some discussion amongst the pandas developers to add this feature of selecting column names with spaces in the library, but it has yet to be built.

## Slicing with just the brackets

So far, we have covered three ways to select subsets of data with just the brackets. With a single string, a list of strings, and a boolean Series. Let's quickly review those ways right now.

### A single string

In [ ]:
bikes['tripduration'].head(3)

### A list of strings

In [ ]:
cols = ['trip_id', 'tripduration']
bikes[cols].head(3)

### A boolean Series
The previous two examples selected columns. Boolean Series select rows.

In [ ]:
filt = bikes['tripduration'] > 5000
bikes[filt].head(3)

### Using a slice
It is possible to use slice notation within just the brackets. For example, the following selects the rows beginning at location 2 up to location 10 with a step size of 4. You can even use slice notation when the index is strings.

In [ ]:
bikes[2:10:4]

### I do not recommend using slicing with *just the brackets*

Although slicing with *just the brackets* seems simple, I do not recommend using it. This is because it is ambiguous and can make selections either by integer location or by label. I always prefer explicit, unambiguous methods. Both `loc` and `iloc` are unambiguous and explicit. Meaning that even without knowing anything about the DataFrame, you would be able to explain exactly how the selection will take place.

If you do want to slice the rows, then use `loc` if you are using labels and `iloc` if you are using integer location, but do not use *just the brackets*.

## Selecting a single cell with `at` and `iat`

pandas provides two more rarely seen indexers, `at`, and `iat`. These indexers are analogous to `loc` and `iloc` respectively, but only select a single cell of a DataFrame. Since they only select a single cell, you must pass both a row and column selection as either a label (`loc`) or an integer location (`iloc`). Let's see an example of each.

In [ ]:
bikes.at[40, 'temperature']

In [ ]:
bikes.iat[-30, 5]

The current index labels for `bikes` is integers which is why the number 40 was used above. It is the label for a row, but also happens to be an integer.

### What's the purpose of these indexers?
All usages of `at` and `iat` may be replaced with `loc` and `iloc` in your data analysis and the code would produce the exact same results. Let's verify this below.

In [ ]:
bikes.loc[40, 'temperature']

In [ ]:
bikes.iloc[-30, 5]

These `at` and `iat` indexers are optimized to select a single cell of data and therefore provide slightly better performance than `loc` or `iloc`.

### I never use these indexers

Personally, I never use these specialty indexers as the performance advantage for a single selection is minor. It would require a case where single element selections were happening in great numbers to see any significant improvement and doing so is rare in data analysis.

### Much bigger performance improvement using numpy directly

If you truly wanted a large performance improvement for single-cell selection, you would select directly from numpy arrays and not a pandas DataFrame. Below, the data is extracted into the underlying numpy array with the `values` attribute. We then time the performance for selecting with numpy and also with `iat` and `iloc` on a DataFrame. On my machine, `iat` shows a 30-40% improvement over `iloc`, but selecting with numpy is about 50x as fast. There is no comparison here, if you care about performance for selecting a single cell of data, use numpy.

In [ ]:
values = bikes.values

In [ ]:
%timeit -n 5 values[-30, 5]

In [ ]:
%timeit -n 5 bikes.iat[-30, 5]

In [ ]:
%timeit -n 5 bikes.iloc[-30, 5]

## Exercises

### Exercise 1
<span  style="color:green; font-size:16px">Use the `query` method to select trip durations between 5000 and 10000 when it was partlycloudly or mostlycloudly. Create a set to contain the possible events and assign it to a variable. Reference this variable within the query string. Then, redo the operation again using boolean selection.</span>